<a href="https://colab.research.google.com/github/wendy60/Hybrid-recommender-system/blob/initial-user-interest-model/LSTUR_Neural_News_Recommendation_with_Long_and_Short_term_User_Representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Global settings and imports**

In [ ]:
pip install scrapbook

     |████████████████████████████████| 150 kB 9.2 MB/s 
     |████████████████████████████████| 125 kB 41.1 MB/s 
     |████████████████████████████████| 743 kB 33.8 MB/s 
     |████████████████████████████████| 749 kB 28.9 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires typing-extensions~=

In [ ]:
pip install recommenders

     |████████████████████████████████| 314 kB 5.3 MB/s 
     |████████████████████████████████| 12.4 MB 33.4 MB/s 
     |████████████████████████████████| 310 kB 51.4 MB/s 
     |████████████████████████████████| 1.5 MB 49.9 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 11.8 MB 25.7 MB/s 
     |████████████████████████████████| 93 kB 2.1 MB/s 
     |████████████████████████████████| 636 kB 41.9 MB/s 
     |████████████████████████████████| 61 kB 8.8 MB/s 
     |████████████████████████████████| 3.1 MB 40.5 MB/s 
     |████████████████████████████████| 895 kB 48.0 MB/s 
     |████████████████████████████████| 59 kB 7.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.0 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705381 sha256=636ca12dc1d4dd48c0a9f24f46125d0f45f8da7e2ac0d075902778c74f23985c
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b6

In [ ]:
pip install tensorflow-gpu==1.15.2

     |████████████████████████████████| 410.9 MB 33 kB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 3.8 MB 39.5 MB/s 
     |████████████████████████████████| 503 kB 37.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=4d3f65a5d55b96383642531416bccf18efd3079ba753fe0a174f33f419ed7021
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.lstur import LSTURModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


# **Prepare Parameters**

In [ ]:
epochs = 5
seed = 40
batch_size = 32

# Options: demo, small, large
MIND_type = 'demo'

# **Download and load data**

In [ ]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)


100%|██████████| 17.0k/17.0k [00:00<00:00, 24.7kKB/s]
100%|██████████| 9.84k/9.84k [00:00<00:00, 21.8kKB/s]
100%|██████████| 95.0k/95.0k [00:02<00:00, 35.5kKB/s]



# **Create hyper-parameters**

In [ ]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

data_format=news,iterator_type=None,support_quick_scoring=True,wordEmb_file=/tmp/tmpmqcgys4b/utils/embedding.npy,wordDict_file=/tmp/tmpmqcgys4b/utils/word_dict.pkl,userDict_file=/tmp/tmpmqcgys4b/utils/uid2index.pkl,vertDict_file=None,subvertDict_file=None,title_size=30,body_size=None,word_emb_dim=300,word_size=None,user_num=None,vert_num=None,subvert_num=None,his_size=50,npratio=4,dropout=0.2,attention_hidden_dim=200,head_num=4,head_dim=100,cnn_activation=relu,dense_activation=None,filter_num=400,window_size=3,vert_emb_dim=100,subvert_emb_dim=100,gru_unit=400,type=ini,user_emb_dim=50,learning_rate=0.0001,loss=cross_entropy_loss,optimizer=adam,epochs=5,batch_size=32,show_step=100000,metrics=['group_auc', 'mean_mrr', 'ndcg@5;10']


In [ ]:
iterator = MINDIterator

# **Train the LSTUR model**

In [ ]:
model = LSTURModel(hparams, iterator, seed=seed)

Tensor("conv1d/Relu:0", shape=(?, 30, 400), dtype=float32)
Tensor("att_layer2/Sum_1:0", shape=(?, 400), dtype=float32)


In [ ]:

print(model.run_eval(valid_news_file, valid_behaviors_file))

586it [00:04, 120.38it/s]
236it [00:07, 29.88it/s]
7538it [00:01, 4941.27it/s]


{'group_auc': 0.5201, 'mean_mrr': 0.2214, 'ndcg@5': 0.2292, 'ndcg@10': 0.2912}


In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

1086it [02:38,  6.84it/s]
586it [00:01, 553.14it/s]
236it [00:07, 30.91it/s]
7538it [00:01, 5169.79it/s]


at epoch 1
train info: logloss loss:1.486071678046583
eval info: group_auc:0.5989, mean_mrr:0.2609, ndcg@10:0.3495, ndcg@5:0.2861
at epoch 1 , train time: 158.7 eval time: 19.3


1086it [02:31,  7.18it/s]
586it [00:01, 564.08it/s]
236it [00:07, 32.36it/s]
7538it [00:01, 5073.90it/s]


at epoch 2
train info: logloss loss:1.401073770847988
eval info: group_auc:0.6146, mean_mrr:0.2751, ndcg@10:0.3657, ndcg@5:0.2992
at epoch 2 , train time: 151.3 eval time: 18.9


1086it [02:29,  7.26it/s]
586it [00:01, 545.61it/s]
236it [00:07, 30.47it/s]
7538it [00:01, 5435.15it/s]


at epoch 3
train info: logloss loss:1.3540291745579265
eval info: group_auc:0.6362, mean_mrr:0.2915, ndcg@10:0.3848, ndcg@5:0.3228
at epoch 3 , train time: 149.6 eval time: 19.2


1086it [02:28,  7.32it/s]
586it [00:01, 560.91it/s]
236it [00:07, 30.82it/s]
7538it [00:01, 5558.78it/s]


at epoch 4
train info: logloss loss:1.3227735883513427
eval info: group_auc:0.6377, mean_mrr:0.2865, ndcg@10:0.3823, ndcg@5:0.318
at epoch 4 , train time: 148.4 eval time: 18.9


1086it [02:26,  7.42it/s]
586it [00:01, 569.08it/s]
236it [00:07, 32.70it/s]
7538it [00:01, 5478.34it/s]


at epoch 5
train info: logloss loss:1.2843476580541657
eval info: group_auc:0.6428, mean_mrr:0.2985, ndcg@10:0.3929, ndcg@5:0.3314
at epoch 5 , train time: 146.4 eval time: 18.3
CPU times: user 17min 49s, sys: 1min 41s, total: 19min 30s
Wall time: 14min 8s


In [ ]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

586it [00:01, 535.29it/s]
236it [00:07, 31.44it/s]
7538it [00:01, 5255.15it/s]


{'group_auc': 0.6428, 'mean_mrr': 0.2985, 'ndcg@5': 0.3314, 'ndcg@10': 0.3929}
CPU times: user 21.1 s, sys: 2.81 s, total: 23.9 s
Wall time: 19 s


In [ ]:
sb.glue("res_syn", res_syn)

# **Save the model**

In [ ]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "lstur_ckpt"))

# **Output Prediction File**

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)


586it [00:01, 517.97it/s]
236it [00:08, 29.44it/s]
7538it [00:01, 5192.91it/s]


In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')


7538it [00:00, 44934.16it/s]


In [ ]:

f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()